In [1]:
from model.custom_transformer import *

In [2]:
from model.patch_model import *

In [3]:
model = POverLapTimePosVallinaViT((14,32,64),(3,5,5),in_chans=5)

for 2D patch model, the img_size will be force set (14, 32, 64)


In [4]:
from train.pretrain import *

In [5]:
args = get_args(config_path="checkpoints/WeathBench7066PatchDataset/ViT_in_bulk-POverLapTimePosVallinaViT_Patch_(3, 5, 5)/time_step_2_pretrain-3D70N_every_1_step_random_dataset/12_07_22_50_36-seed_2018/config.json")

In [6]:
time_step = args.time_step if "fourcast" in args.mode else 3*24//6 + args.time_step
dataset_kargs = copy.deepcopy(args.dataset_kargs)
dataset_kargs['time_step'] = time_step
if dataset_kargs['time_reverse_flag'] in ['only_forward','random_forward_backward']:
    dataset_kargs['time_reverse_flag'] = 'only_forward'
dataset_type = eval(args.dataset_type) if isinstance(args.dataset_type,str) else args.dataset_type
#print(dataset_kargs)
split = args.split if hasattr(args,'split') and args.split else "test"
test_dataset = dataset_type(split=split, with_idx=True,dataset_tensor=None,record_load_tensor=None,**dataset_kargs)

use offline data mode <1>: only train use offline data
use dataset in datasets/weatherbench_6hour
load data from datasets/weatherbench_6hour/test.npy
notice we will use around_index(12, 28, 64, 3, 3, 5, 5) to patch data


In [18]:
test_dataset.cross_sample = False
idx_a,batch = test_dataset[0]
tensor_a, time_stamp_a, pos_a = batch[0]
input_a = model.image_to_patches(tensor_a[None],pos_a)

In [29]:
x = tensor_a[None]
good_input_shape = model.patch_range
now_input_shape  = tuple(x.shape[-3:])

around_index = model.around_index_depend_input(now_input_shape)
if model.global_position_feature is None:
    grid = torch.Tensor(np.stack(np.meshgrid(
            np.arange(model.img_size[0]),
            np.arange(model.img_size[1]),
            np.arange(model.img_size[2]))).transpose(0,2,1,3))
    model.global_position_feature = model.get_direction_from_stamp(grid[None]) #(1, 3, Z,W, H)
pos_feature  = model.global_position_feature.repeat(x.size(0),1,1,1,1).to(x.device) #(B, 3, Z,W, H)
B,P,_,_,_ = x.shape
x = torch.cat([x,pos_feature],1) #( B, 77, W, H)
# x = x[...,around_index[:,:,:,0],around_index[:,:,:,1],around_index[:,:,:,2]] # (B,P,Z,W-4,H,Patch,Patch)
# B,_,Z,W,H,_,_,_ = x.shape
# model.input_shape_tmp=(B,Z,W,H,P)

In [32]:
pos_b.shape

(3, 3, 5, 5)

In [41]:
test_dataset.cross_sample = True
idx_b,batch = test_dataset[0]
(tensor_b, time_stamp_b, pos_b) = batch[0]
input_b = model.image_to_patches(tensor_b[None],torch.Tensor(pos_b[None]).to(tensor_b.device))
print(pos_b)

[[[[10 10 10 10 10]
   [10 10 10 10 10]
   [10 10 10 10 10]
   [10 10 10 10 10]
   [10 10 10 10 10]]

  [[11 11 11 11 11]
   [11 11 11 11 11]
   [11 11 11 11 11]
   [11 11 11 11 11]
   [11 11 11 11 11]]

  [[12 12 12 12 12]
   [12 12 12 12 12]
   [12 12 12 12 12]
   [12 12 12 12 12]
   [12 12 12 12 12]]]


 [[[19 19 19 19 19]
   [20 20 20 20 20]
   [21 21 21 21 21]
   [22 22 22 22 22]
   [23 23 23 23 23]]

  [[19 19 19 19 19]
   [20 20 20 20 20]
   [21 21 21 21 21]
   [22 22 22 22 22]
   [23 23 23 23 23]]

  [[19 19 19 19 19]
   [20 20 20 20 20]
   [21 21 21 21 21]
   [22 22 22 22 22]
   [23 23 23 23 23]]]


 [[[18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]]

  [[18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]]

  [[18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]]]]


In [42]:
selected_tensor= x[...,pos_b[0],pos_b[1],pos_b[2]]
torch.dist(selected_tensor,input_b)

tensor(0., dtype=torch.float64)

In [19]:
grid = torch.Tensor(np.stack(np.meshgrid(
                np.arange(model.img_size[0]),
                np.arange(model.img_size[1]),
                np.arange(model.img_size[2]))).transpose(0,2,1,3))

In [23]:
a = torch.randn(1,5,14,32,64)
y = model.image_to_patches(a)
c = model.patches_to_image(y)

TypeError: image_to_patches() missing 1 required positional argument: 'pos_stamp'

In [20]:
grid.shape

torch.Size([3, 14, 32, 64])

In [8]:
a = torch.randn(1,70,3,5,5).cuda()
p = torch.randn(1, 3,3,5,5).cuda()
t = torch.randn(1,4).cuda()
model = model.cuda()

In [9]:
model(a,t,p).shape

torch.Size([1, 70, 3, 5, 5])

In [7]:
import numpy as np

In [18]:
torch.Tensor(np.stack(np.meshgrid(
            np.arange(img_size[0]),
            np.arange(img_size[1]),
        )).transpose(0,2,1)).shape

torch.Size([2, 28, 32])

In [19]:
torch.Tensor(np.stack(np.meshgrid(
    np.arange(self.img_size[0]),
    np.arange(self.img_size[1]),
    np.arange(self.img_size[2]))).transpose(0,2,1,3)).shape


torch.Size([3, 28, 32, 36])

In [4]:
model(a).shape

torch.Size([1, 70, 14, 32, 64])

In [6]:
32*16*7

3584

In [101]:
a = torach.randn(1,2,4,4)
m = torch.randint(2,(4,4))

In [102]:
a.masked_fill(m,-1)

tensor([[[[-1.0000, -1.0000,  1.2643,  0.2839],
          [-1.0000,  0.0204, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -0.7041],
          [ 0.1266, -1.0000, -1.0000, -1.0000]],

         [[-1.0000, -1.0000, -2.2420, -0.1599],
          [-1.0000,  1.4979, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -0.3242],
          [ 0.3754, -1.0000, -1.0000, -1.0000]]]])

In [2]:
import numpy as np

In [ ]:

#x = x.reshape(B,W,H,P,patch_range,patch_range)
x = torch.randn(B,W,H,P,patch_range,patch_range)
def get_x1(x,counting_matrix):
    w_idx   = np.arange(0,L)
    wes  = np.stack([w_idx, w_idx+1,w_idx+2, w_idx-1, w_idx-2],1)%L
    x_idx   = np.arange(H)
    xes = np.stack([x_idx, x_idx+1,x_idx+2, x_idx-1, x_idx-2],1)%H
    yes = np.array([[2,  1,  0,  3,  4]])
    x = torch.nn.functional.pad(x,(0,0, 0,0, 0,0, 0,0, 2 , 2 )) # only extend W
    x     = x[:, wes, :,:,yes,:].sum(1) #(B, W, H, P, PS,PS) --> (W, B, H, P, PS)
    x     = x[:, :, xes,:,yes].sum(1)  #(W, B, H, P, PS) --> (H, W, B, P)   
    x     = x.permute(2,3,1,0)#(B,P, W,H)   
    counting_matrix =counting_matrix.to(x.device)
    x     = x/counting_matrix #(B,P, W,H)  / (1,1 , W,H)
    return x
def get_x2(x,counting_matrix):
    w_idx   = np.arange(0,L)
    wes  = np.stack([w_idx+2, w_idx+1, w_idx, w_idx-1, w_idx-2],1)%L
    x_idx   = np.arange(H)
    xes = np.stack([x_idx+2, x_idx+1, x_idx, x_idx-1, x_idx-2],1)%H
    yes = np.array([[0 ,1, 2, 3, 4]])
    x = torch.nn.functional.pad(x,(0,0, 0,0, 0,0, 0,0, 2 , 2 )) # only extend W
    x     = x[:, wes, :,:,yes,:].sum(1) #(B, W, H, P, PS,PS) --> (W, B, H, P, PS)
    x     = x[:, :, xes,:,yes].sum(1)  #(W, B, H, P, PS) --> (H, W, B, P)   
    x     = x.permute(2,3,1,0)#(B,P, W,H)   
    counting_matrix =counting_matrix.to(x.device)
    x     = x/counting_matrix #(B,P, W,H)  / (1,1 , W,H)
    return x
print(torch.dist(get_x1(x,counting_matrix),get_x2(x,counting_matrix)))

In [ ]:
from vit_pytorch.vit import repeat

In [ ]:
from mltool.universal_model_util import get_model_para_num
from utils.tools import getModelSize

In [ ]:
from model.othermodels import *

In [ ]:
class CK_SWDformer_0505(BaseModel):
        def __init__(self,*args,**kargs):
            super().__init__()
            print("this is pre-set model, we disable all config")
            self.backbone = SWD_former(patch_size= [1, 1],
                                in_chans= 70,
                                out_chans= 70,
                                embed_dim= 768,
                                window_size= (5,5),
                                depths= [4, 4, 4],
                                num_heads= [6, 6, 6],
                                Weather_T=1,only_swin=True)
        def forward(self,x):
            return self.backbone(x)

In [ ]:
model = CK_SWDformer_0505()

In [ ]:
a=torch.randn(1,70,5,5)
model(a).shape

In [ ]:
W=32
H=64
hd=16
D=768
B=3

In [ ]:
from networks.SWD_former import SWD_former,SWDfromer

In [ ]:
model = SWD_former(patch_size= [1, 1],
        in_chans= 70,
        out_chans= 70,
        embed_dim= 1128,
        window_size= (32,64),
        depths= [4, 4, 4],
        num_heads= [6, 6, 6],
        Weather_T=1,only_swin=True)

In [ ]:
from mltool.universal_model_util import get_model_para_num

In [ ]:
from model.afnonet import AFNONet

In [ ]:
get_model_para_num(model)[1]/1024/1024

In [ ]:
import torch

In [ ]:
import numpy as np

In [ ]:
from cephdataset import WeathBench7066PatchDataset

In [ ]:
img = torch.randn(1,70,32,64)
x = model.backbone.to_patch_embedding(img);print(x.shape)
b, n, _ = x.shape

cls_tokens = repeat(model.backbone.cls_token, '1 1 d -> b 1 d', b = b)
x = torch.cat((cls_tokens, x), dim=1);print(x.shape)
x += model.backbone.pos_embedding[:, :(n + 1)];print(x.shape)
x = model.backbone.dropout(x);print(x.shape)

x = model.backbone.transformer(x);print(x.shape)

x = x.mean(dim = 1) if model.backbone.pool == 'mean' else x[:, 0];print(x.shape)

x = model.backbone.to_latent(x);print(x.shape)

In [ ]:
property_names = test_dataset.vnames
accu_list = torch.stack([p['accu'] for p in fourcastresult.values() if 'accu' in p]).numpy()
total_num = len(accu_list)
accu_list = accu_list.mean(0)# (fourcast_num,property_num)
real_times = [(predict_time+1)*test_dataset.time_intervel*test_dataset.time_unit for predict_time in range(len(accu_list))]

In [ ]:
snap_tables=[]
snap_index = fourcastresult['snap_index']
select_snap_start_time_point = snap_index[0]
select_snap_show_property_id = snap_index[1]
select_snap_property_name    = [property_names[iidd] for iidd in select_snap_show_property_id]
for select_time_point in select_snap_start_time_point:
    timestamp = test_dataset.datatimelist_pool['test'][select_time_point]
    if select_time_point in fourcastresult: # in case do not record
        linedata = fourcastresult[select_time_point]['snap_line']
        for predict_time_point, tensor, label in linedata:
            for name, value in zip(select_snap_property_name,tensor):
                predict_timestamp = 0 if predict_time_point==0 else real_times[predict_time_point-1]
                snap_tables.append([timestamp, name, predict_timestamp, value.item(), label])

In [ ]:
global_rmse_map = fourcastresult['global_rmse_map']
mean_global_rmse_map = [t/total_num for t in global_rmse_map]

In [ ]:
import numpy 

In [ ]:
import pandas as pd

# Old

In [ ]:
!pip install empatches

In [ ]:
from empatches import EMPatches

In [ ]:
emp = EMPatches()

In [ ]:
img_patches, indices = emp.extract_patches(img, patchsize=512, overlap=0.2)

In [ ]:
line.shape

In [ ]:
yes.shape

In [ ]:
print(topvalues)

In [ ]:
for i in range(64):
    [i,]

In [ ]:
patch_x_idx

In [ ]:
for i in range()

In [ ]:
a = torch.randn(23,70,32,64)

In [ ]:
batch_idx  = np.array([1,2])
proper_idx = np.array([3,4]) 

In [ ]:
a[0,0][2,1]

In [ ]:
a[0,0][[2,3,4],[1,2,3]]

In [ ]:
a.std(dim=(1,2,3)).shape

In [ ]:
t = torch.tensor([[1, 2], [3, 4]])
torch.gather(t, 1, torch.tensor([[0], [1]]))

In [ ]:
pos = [0,1,2,3,4]

In [ ]:
a = np.random.randn(1,2,3,4,5)

In [ ]:
a[*pos]

In [ ]:
import numpy as np

In [ ]:
a = np.random.randn(1500,2000)

In [ ]:
a.nbytes//8//1024//1024

In [ ]:
from model.patch_model import *

In [ ]:
from model.afnonet import AFNONet

In [ ]:
import torch

In [ ]:
from mltool.universal_model_util import get_model_para_detail

In [ ]:
class LargeMLP_3D(nn.Module):
    '''
    input is (B, P, patch_range_1,patch_range_2,patch_range_3)
    output is (B,P)
    ''' 
    def __init__(self,img_size=None,patch_range=5,in_chans=20, out_chans=20,p=0.1,**kargs):
        super().__init__()
        self.img_size = img_size
        self.patch_range = 5
        if self.patch_range == 5:
            cl = [5*5*5*in_chans,5*5*5*10,5*5*5*20,
                  5*5*5*30,5*5*5*30,5*5*5*20,
                  5*5*5*10,5*5*5*1,out_chans]
            nnlist = []
            for i in range(len(cl)-2):
                nnlist+=[nn.Linear(cl[i],cl[i+1]),nn.Dropout(p=p),nn.Tanh()]
            nnlist+=[nn.Linear(cl[-2],cl[-1])]
            self.backbone = nn.Sequential(*nnlist)
        else:
            raise NotImplementedError
        self.center_index,self.around_index=get_center_around_indexes_3D(self.patch_range,self.img_size)

    def forward(self, x):
        '''
        The input either (B,P,patch_range,patch_range) or (B,P,w,h)
        The output then is  (B,P) or (B,P,w-patch_range//2,h-patch_range//2)
        ''' 
        assert len(x.shape) == 5 #(B,P,Z,W,H)
        input_is_full_image = False
        if x.shape[-3:] == self.img_size:
            input_is_full_image = True
            x = x[..., self.around_index[:, :, : , 0],self.around_index[:, :, : , 1],self.around_index[:, :, : , 2]] 
            # (B,P,Z-2,W-2,H,Patch,Patch,Patch)
            x = x.permute(0, 2, 3, 4, 1, 5, 6, 7)
            B, Z, W, H, P, _, _, _ = x.shape
            x = x.flatten(0, 3)  # (B* Z-2 * W-2 * H, Property, Patch,Patch,Patch)
        assert tuple(x.shape[-3:]) == (self.patch_range,self.patch_range,self.patch_range)
        x = self.backbone(x.flatten(-4,-1)) # (B* W-4 * H,P)
        if input_is_full_image:
            x = x.reshape(B, Z, W, H, P).permute(0, 4, 1, 2,3) #(B, Z-2,W-2,H,P)  -> (B,P, Z-2,W-2,H)
        return x


In [ ]:
model = LargeMLP_3D((14,32,64))

In [ ]:
from utils.tools import getModelSize
param_sum, buffer_sum, all_size = getModelSize(model)
print(f" Number of Parameters: {param_sum}, Number of Buffers: {buffer_sum}, Size of Model: {all_size:.4f} MB\n")

In [ ]:
from model.patch_model import NaiveConvModel2D

In [ ]:
import torch

In [ ]:
model = NaiveConvModel2D((32,64))

In [ ]:
a = torch.randn(1,20,32,64)

In [ ]:
from utils.tools import get_center_around_indexes

In [ ]:
img_shape=(32,64)
patch_range=5
center_index,around_index=get_center_around_indexes(patch_range,img_shape)

In [ ]:
around_index.shape

In [ ]:
patch_range=5
delta = [list(range(-(patch_range//2),patch_range//2+1))]*2
delta = np.meshgrid(*delta)

In [ ]:
delta[0].shape

In [ ]:
delta = [list(range(-(patch_range//2),patch_range//2+1))]*len(center)
delta = np.meshgrid(*delta)
pos  = [c+dc for c,dc in zip(center,delta)]
pos[-1]= pos[-1]%64

px = x + dx
py = y + dy
np.stack([px.flatten(),py.flatten()])

In [ ]:
import torch

In [ ]:
coordinate = np.arange(36).reshape(1,1,6,6)
print(coordinate)

In [ ]:
coordinate[:,:,pos].shape

In [ ]:
print(coordinate[:,:,x,y][0,0])
print(coordinate[:,:,pos[0],pos[1]][0,0])

In [ ]:
from model.FEDformer import *

In [ ]:
nn.BatchNorm3d()

In [ ]:
class moving_avg_spacetime(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super().__init__()
        assert len(kernel_size)==3
        self.kernel_size = np.array(kernel_size)
        self.avg         = nn.AvgPool3d(kernel_size=kernel_size, stride=stride, padding=0)
        self.pad_front   = self.kernel_size - 1-np.floor((self.kernel_size - 1) // 2)
        self.pad_end     = np.floor((self.kernel_size - 1) // 2)
        self.pad         = np.stack([self.pad_front,self.pad_end],1)[::-1].flatten().astype('int').tolist()
        print(self.pad)
    def forward(self, x):
        # padding on the both ends of time series
        # the input must be (B,*Space,T,C)， the -1 dim is embed channel, the -2 dim is time channel
        shape = x.shape
        BSpace_shape = shape[:-2]
        C = shape[-1]
        permute_order = [0,-1] + list(range(1,len(shape)-1))
        x = x.permute(*permute_order)#-->(B, *Space,T, C)-->(B, C, *Space,T)
        x = self.avg(F.pad(x,self.pad, mode='replicate'))
        permute_order = [0] + list(range(2,len(shape))) + [1]
        x = x.permute(*permute_order)
        return x

In [ ]:
layer = moving_avg_spacetime((5,3,2),1)

In [ ]:
a=torch.randn(1,32,64,6,7)
layer(a).shape

In [ ]:
from model.physics_model import *

In [ ]:
from model.afnonet import *

In [ ]:
from utils.params import *

In [ ]:
args=get_args(args=[])

In [ ]:
backbone = AFNONet((32,64),2,265,20,depth=1)

In [ ]:
layer = DirectSpace_Feature_Model(args,backbone)

In [ ]:
a= torch.randn(1,4,5,32,64)

In [ ]:
torch.std_mean(layer(a))

In [ ]:
layer = Second_Derivative_Layer()

a=torch.randn(1,1,32,64)

layer(a).shape

In [ ]:
B=2
P=4
a=torch.randn(B,P,3,32,64).cuda()
layer=  First_Derivative_Layer(dim=3).cuda()
runtime_weight=layer.runtime_weight

In [ ]:
%%timeit
x = torch.conv3d(a.flatten(0,1).unsqueeze(1),runtime_weight).reshape(*a.shape[:-1],-1)

In [ ]:
%%timeit
x2 = torch.conv1d(a.flatten(0,-2).unsqueeze(1),runtime_weight[0,0]).reshape(*a.shape[:-1],-1)

In [ ]:
torch.dist(x,x2)

In [ ]:
%matplotlib inline
plt.scatter(x,y)
#plt.scatter([10],[10],'r')

In [ ]:
def get_fourier_drive(x, position,pad):

    if len(position)==1:position = position[0]
    base_len          = np.array(x.shape[position])
    interpolate_shape = base_len + (base_len-1)*pad
    interpolate_shape = tuple(interpolate_shape)
    if isinstance(position,int) or len(position) == 1
        x = x.transpose(-1,position)
        out_shape = x.shape[:-1]
        x = x.reshape(x.size(0),-1,x.size(-1))
        b = torch.nn.functional.interpolate(x,interpolate_shape,mode='linear',align_corners=True)
        bf = torch.fft.rfft(b,dim=-1)
        kbf= bf*torch.fft.rfftfreq(b.size(-1)).reshape(1,1,-1)
        kb = torch.fft.irfft(kbf,dim=-1)
        index = torch.arange(0,b.size(-1)+1,base_len-1)
        kb = kb[...,index]

In [ ]:
#from tkinter.messagebox import NO
import numpy as np
import torch,os,io,socket
from torchvision import datasets, transforms
hostname = socket.gethostname()
from functools import lru_cache
import traceback
from tqdm import tqdm
import pandas as pd
from utils.timefeatures import time_features
import os
import h5py


In [ ]:
datatimelist  = np.arange(np.datetime64("1979-01-01"), np.datetime64("2016-01-01"), np.timedelta64(6, "h"))
timestamp = time_features(pd.to_datetime(datatimelist)).transpose(1, 0)


In [ ]:
pd.to_datetime(np.datetime64("1980-12-31")).dayofyear

In [ ]:
pd.to_datetime(datatimelist)[0].dayofyear

In [ ]:
timestamp[:10,[0,-1]]

In [ ]:
from JCmodels.fourcastnet import *

In [ ]:
model = AFNONet((32,64))

In [ ]:
a = torch.randn(1,4,5,32,64)

In [ ]:
model(a).shape

#### FEDformer

#### complex version AdaptiveFourierNeuralOperator

##### validate

In [ ]:
def multiply(input, weights):
    return torch.einsum('...bd,bdk->...bk', input, weights)

In [ ]:
B=1
h=32;w=64;C=2
num_blocks=1
block_size=2

w1 = torch.randn(num_blocks, block_size, block_size, dtype=torch.cfloat)
b1 = torch.randn(num_blocks, block_size, dtype=torch.cfloat)
w2 = torch.randn(num_blocks, block_size, block_size, dtype=torch.cfloat)
b2 = torch.randn(num_blocks, block_size, dtype=torch.cfloat)

In [ ]:
def original_realize(x,w1,b1,w2,b2):
    x = torch.fft.rfft2(x, dim=(1, 2), norm='ortho');
    x = x.reshape(B, x.shape[1], x.shape[2], num_blocks, block_size)
    x_real = F.relu(multiply(x.real, w1.real) - multiply(x.imag, w1.imag) + b1.real, inplace=True)
    x_imag = F.relu(multiply(x.real, w1.imag) + multiply(x.imag, w1.real) + b1.imag, inplace=True)
    x = torch.stack([x_real, x_imag], dim=-1)
    x = torch.view_as_complex(x)
    return x

def complex_version_realize(x,w1,b1,w2,b2):
    x = torch.fft.rfft2(x, dim=(1, 2), norm='ortho');
    x = x.reshape(B, x.shape[1], x.shape[2], num_blocks, block_size)
    x = multiply(x,w1)+b1
    x = nonlinear_activate(x)
    return x
x  = torch.randn(B, h, w, C)
y1 = original_realize(x,w1,b1,w2,b2)
y2 = complex_version_realize(x,w1,b1,w2,b2)
torch.dist(y1,y2)

In [ ]:
from model.afnonet import *

##### AFNONet3D

In [ ]:
class PartReLU_Complex(nn.Module):
    def forward(self,x):
        F.relu(x.real, inplace=True)
        F.relu(x.imag, inplace=True)
        return x

class AdaptiveFourierNeuralOperator(nn.Module):
    def __init__(self, dim, img_size, fno_blocks=4,fno_bias=True, fno_softshrink=False,nonlinear_activate=PartReLU_Complex()):
        super().__init__()

        self.hidden_size = dim
        self.img_size   = img_size
        self.num_blocks = fno_blocks
        self.block_size = self.hidden_size // self.num_blocks
        assert self.hidden_size % self.num_blocks == 0

        self.scale = 0.02
        self.w1 = torch.nn.Parameter(self.scale * torch.randn(self.num_blocks, self.block_size, 
                                                              self.block_size, dtype=torch.cfloat))
        self.b1 = torch.nn.Parameter(self.scale * torch.randn(self.num_blocks, self.block_size, 
                                                              dtype=torch.cfloat))
        self.w2 = torch.nn.Parameter(self.scale * torch.randn(self.num_blocks, self.block_size, 
                                                              self.block_size, dtype=torch.cfloat))
        self.b2 = torch.nn.Parameter(self.scale * torch.randn(self.num_blocks, self.block_size, 
                                                              dtype=torch.cfloat))
        self.relu = nonlinear_activate

        if fno_bias:
            self.bias = nn.Conv1d(self.hidden_size, self.hidden_size, 1)
        else:
            self.bias = None

        self.softshrink = fno_softshrink

    def multiply(self, input, weights):
        return torch.einsum('...bd,bdk->...bk', input, weights)

    def forward(self, x):
        B, N, C = x.shape    
        bias = self.bias(x.permute(0, 2, 1)).permute(0, 2, 1) if self.bias else 0
        #timer.restart(2)
        x = x.reshape(B, *self.img_size, C)
        fft_dim = tuple(range(1,len(self.img_size)+1))
        print(fft_dim)
        #timer.record('reshape1','filter',2)
        print(x.shape)
        x = torch.fft.rfftn(x, dim=fft_dim, norm='ortho');
        #timer.record('rfft2','filter',2)
        print(x.shape)
        x = x.reshape(*x.shape[:-1], self.num_blocks, self.block_size)
        #timer.record('reshape2','filter',2)
        x = self.multiply(x,self.w1)+self.b1
        x = self.relu(x)
        x = self.multiply(x,self.w2)+self.b2
        x = F.softshrink(x, lambd=self.softshrink) if self.softshrink else x
        #with torch.cuda.amp.autocast(enabled=False):
        #x = x.float()   
        #x = torch.view_as_complex(x)
        #timer.record('reset','filter',2)
        x = x.flatten(-2,-1)
        #timer.record('reshape3','filter',2)
        print(x.shape)
        x = torch.fft.irfftn(x, s=self.img_size,dim=fft_dim, norm='ortho')
        print(x.shape)
        #x = x.half()
        #timer.record('irfft2','filter',2)
        x = x.reshape(B, N, C)
        #timer.record('reshape4','filter',2)
        return x + bias

class Block(nn.Module):
    def __init__(self, dim, mlp_ratio=4., drop=0., drop_path=0., act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm, region_shape=(14,8), fno_blocks=3,double_skip=False, fno_bias=False, fno_softshrink=False):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.filter = AdaptiveFourierNeuralOperator(dim, region_shape,fno_blocks=fno_blocks,fno_bias=fno_bias,fno_softshrink=fno_softshrink)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
        self.double_skip = double_skip

    def forward(self, x):
        residual = x
        #timer.restart(1)
        x = self.norm1(x)
        #timer.record('norm1','forward_features',1)
        x = self.filter(x)
        #timer.record('filter','forward_features',1)
        if self.double_skip:
            x += residual
            residual = x;
        #timer.record('residual','forward_features',1)
        x = self.norm2(x)
        #timer.record('norm2','forward_features',1)
        x = self.mlp(x)
        #timer.record('mlp','forward_features',1)
        x = self.drop_path(x)
        #timer.record('drop_path','forward_features',1)
        x += residual
        #timer.record('residual','forward_features',1)
        return x

class PatchEmbed(nn.Module):
    def __init__(self, img_size=None, patch_size=8, in_chans=13, embed_dim=768):
        super().__init__()

        if img_size is None:raise KeyError('img is None')
        patch_size   = [patch_size]*len(img_size) if isinstance(patch_size,int) else patch_size
        
        num_patches=1
        out_size=[]
        for i_size,p_size in zip(img_size,patch_size):
            if p_size%i_size:
                num_patches*=i_size// p_size
                out_size.append(i_size// p_size)
            else:
                raise NotImplementedError(f"the patch size ({patch_size}) cannot divide the img size {img_size}")
        self.img_size    = tuple(img_size)
        self.patch_size  = tuple(patch_size)
        self.num_patches = num_patches
        self.out_size    = tuple(out_size)
        conv_engine = [nn.Conv1d,nn.Conv2d,nn.Conv3d]
        self.proj   = conv_engine[len(img_size)-1](in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, = x.shape[:2]
        inp_size = x.shape[2:]
        assert tuple(inp_size) == self.img_size, f"Input image size ({inp_size}) doesn't match model set size ({self.img_size})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

class AFNONet(nn.Module):
    """
    
    """
    def __init__(self, img_size, patch_size=8, in_chans=20, out_chans=20, embed_dim=768, depth=12, mlp_ratio=4.,
                 uniform_drop=False, drop_rate=0., drop_path_rate=0., norm_layer=None,
                 dropcls=0, checkpoint_activations=False, fno_blocks=3,double_skip=False,
                 fno_bias=False, fno_softshrink=False,debug_mode=False):
        super().__init__()

        assert img_size is not None
        self.checkpoint_activations=checkpoint_activations
        self.embed_dim = embed_dim
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        self.img_size = img_size
        self.patch_embed = PatchEmbed(img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches      = self.patch_embed.num_patches
        patch_size       = self.patch_embed.patch_size
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        self.final_shape = self.patch_embed.out_size

        if uniform_drop:
            dpr = [drop_path_rate for _ in range(depth)]  # stochastic depth decay rule
        else:
            dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule

        self.blocks = nn.ModuleList([Block(dim=embed_dim, mlp_ratio=mlp_ratio, drop=drop_rate,
                                           drop_path=dpr[i], 
                                           norm_layer=norm_layer,
                                           region_shape=self.final_shape,
                                           double_skip=double_skip,
                                           fno_blocks=fno_blocks,
                                           fno_bias=fno_bias,
                                           fno_softshrink=fno_softshrink) for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        # Representation layer
        # self.num_features = out_chans * img_size[0] * img_size[1]
        # self.representation_size = self.num_features * 8
        # self.pre_logits = nn.Sequential(OrderedDict([
        #     ('fc', nn.Linear(embed_dim, self.representation_size)),
        #     ('act', nn.Tanh())
        # ]))
        conf_list = [{'kernel_size':[],'stride':[],'padding':[]},
                     {'kernel_size':[],'stride':[],'padding':[]},
                     {'kernel_size':[],'stride':[],'padding':[]}]
        conv_set = {8:[[2,2,0],[2,2,0],[2,2,0]],
                    4:[[2,2,0],[3,1,1],[2,2,0]],
                    2:[[3,1,1],[3,1,1],[2,2,0]],
                    1:[[3,1,1],[3,1,1],[3,1,1]],
                   }
        for patch in patch_size:
            for slot in range(len(conf_list)):
                conf_list[slot]['kernel_size'].append(conv_set[patch][slot][0])
                conf_list[slot]['stride'].append(conv_set[patch][slot][1])
                conf_list[slot]['padding'].append(conv_set[patch][slot][2])

        transposeconv_engine = [nn.ConvTranspose1d,nn.ConvTranspose2d,nn.ConvTranspose3d][len(img_size)-1]
        self.pre_logits = nn.Sequential(OrderedDict([
            ('conv1', transposeconv_engine(embed_dim, out_chans*16, **conf_list[0])),
            ('act1', nn.Tanh()),
            ('conv2', transposeconv_engine(out_chans*16, out_chans*4, **conf_list[1])),
            ('act2', nn.Tanh())
        ]))

        # Generator head
        # self.head = nn.Linear(self.representation_size, self.num_features)
        self.head = transposeconv_engine(out_chans*4, out_chans, **conf_list[2])

        if dropcls > 0:
            print('dropout %.2f before classifier' % dropcls)
            self.final_dropout = nn.Dropout(p=dropcls)
        else:
            self.final_dropout = nn.Identity()

        trunc_normal_(self.pos_embed, std=.02)
        self.apply(self._init_weights)
        self.debug_mode=debug_mode
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def forward_features(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        x += self.pos_embed
        x = self.pos_drop(x)

        if not self.checkpoint_activations:
            for blk in self.blocks:
                x = blk(x)
        else:
            x = checkpoint_sequential(self.blocks, 4, x)

        x = self.norm(x).transpose(1, 2);print(x.shape)
        x = torch.reshape(x, [-1, self.embed_dim, *self.final_shape])
        return x

    def forward(self, x):
        ### we assume always feed the tensor (B, p*z, h, w)
        B, P ,h, w = x.shape
        x = x.reshape(B,-1,*self.img_size)
        #timer.restart(level=0)
        x = self.forward_features(x)
        #timer.record('forward_features',level=0)
        x = self.final_dropout(x)
        #timer.record('final_dropout',level=0)
        x = self.pre_logits(x)
        #timer.record('pre_logits',level=0)
        x = self.head(x)
        #timer.record('head',level=0)
        x = x.reshape(B,P,h,w)
        #timer.show_stat()
        #print("============================")

        return x

In [ ]:
model = AFNONet(img_size=[10,32,64],patch_size=(1,8,8),depth=1)

In [ ]:
a = torch.randn(1,20,10,32,64)

In [ ]:
a = torch.randn(1,3,4,32,64).flatten(1,2)
torch.dist(a.reshape(1,-1,4,32,64).reshape(1,12,32,64),a)

In [ ]:
a = torch.randn(1,12,32,64)
torch.dist(a.reshape(1,3,4,32,64).reshape(1,-1,32,64),a)

##### EulerEquationModel2

In [ ]:
class EulerEquationModel2(nn.Module):
    def __init__(self, args, backbone):
        super().__init__()
        self.Dx= First_Derivative_Layer(position=-1, dim=3, mode=2)
        self.Dy= First_Derivative_Layer(position=-2, dim=3, mode=2)
        self.Dz= First_Derivative_Layer(position=-3, dim=3, mode=1)
        self.thermal_factor  = nn.Parameter(torch.randn(3).reshape(1,3,1,1))
        self.alpha = nn.Parameter(torch.randn(1))
        #self.p_list         = nn.Parameter(torch.Tensor([10,8.5,5]).reshape(1,3,1,1),requires_grad=False)
        self.backbone =  backbone
        self.monitor = True
    def forward(self, Field):
        # u^{t+1} &= u^{t} + F_x - \nabla (Vu)  + u \nabla\cdot V - \partial_x\phi\\
        # v^{t+1} &= v^{t} + F_y - \nabla (Vv)  + v \nabla\cdot V - \partial_y\phi\\
        # T^{t+1} &= T^{t} + Q/C_v + \frac{RT}{C_pp}\omega  - \nabla (VT) +T \nabla\cdot V\\
        # \phi^{t+1}&=\phi^{t} + wg  - \nabla (V\phi)+ \phi \nabla\cdot V \\
        # 0&\approx \nabla\cdot V
        # input -> Field  = [u ,v, T, p] --> (Batch, 4, z, y ,x)
        # need generate unknown data [Fx, Fy , Q, W, o]
        b, si_z, i_y, i_x = Field.shape
        s=4
        i_z= si_z//4
        MachineLearningPart = self.backbone(Field).reshape(b, s+1, i_z, i_y, i_x) #(Batch, 5, z, y ,x)
        ExternalForce = MachineLearningPart[:,:4] #(Batch, 4, z, y ,x)
        o = MachineLearningPart[:,4:5] #(Batch, 1, z, y ,x)
        Field = Field.reshape(b, s, i_z, i_y,  i_x) #(Batch, 5, z, y ,x)
        u = Field[:,0:1]#(Batch, 1, z, y ,x)
        v = Field[:,1:2]#(Batch, 1, z, y ,x)
        T = Field[:,2:3]#(Batch, 1, z, y ,x)
        p = Field[:,3:4]#(Batch, 1, z, y ,x)
        V = torch.cat([u,v,o],1)#(Batch, 3, z, y ,x)
        Nabla_cdot_V = (self.Dx(u[:,0]) + self.Dy(v[:,0]) + self.Dz(o[:,0])).unsqueeze(1)#(Batch, 1, z, y ,x)
        Nabla_V_Field= Nabla_cdot_V*Field #(Batch, 4, z, y ,x)
        Vphysics     = torch.stack([V*u,V*v,V*T,V*p],1)#(Batch,4, 3, z, y ,x)
        Vphysics_dx = self.Dx(Vphysics[:,:,0].flatten(0,1)).reshape(Field.shape)#(Batch, 4, z, y ,x)
        Vphysics_dy = self.Dy(Vphysics[:,:,1].flatten(0,1)).reshape(Field.shape)#(Batch, 4, z, y ,x)
        Vphysics_dz = self.Dz(Vphysics[:,:,2].flatten(0,1)).reshape(Field.shape)#(Batch, 4, z, y ,x)
        PhysicsPart = -Vphysics_dx - Vphysics_dy - Vphysics_dz + Nabla_V_Field #(Batch,4,z, y ,x)
        InteractionPart= torch.stack([self.Dx(p[:,0]),
                                      self.Dy(p[:,0]),
                                      self.thermal_factor*T[:,0]*o[:,0]],1)#(Batch,3,z, y ,x)
        InteractionPart= F.pad(InteractionPart,(0,0,0,0,0,0,0,1)) #(Batch,4,z, y ,x)
        Delta_Fd     = ExternalForce + self.alpha*(PhysicsPart + InteractionPart)
        Field        = Field+ Delta_Fd
        constrain    = Nabla_V_Field
        if not self.monitor:
            return Field.flatten(1,2),(constrain**2).mean()
        else:
            return Field.flatten(1,2),(constrain**2).mean(),{"ExternalForceFactor":(ExternalForce**2).mean().item(),
                                                             "PhysicsDrivenFactor":(PhysicsPart**2).mean().item(),
                                                             "InteractionPart":(InteractionPart**2).mean().item(),
                                                             "alpha":self.alpha.item()}
